```
bryan@ichabod mac_bucket % for file in *_vector_log.json; do
  databricks fs cp "$file" dbfs:/Volumes/geodata/petra/vector_log/
done
```

In [ ]:
df = spark.read.format('json').load("/Volumes/geodata/petra/vector_log/")

display(df)

In [ ]:
from common.transforms import string_to_iso_date, generate_hash
import pyspark.sql.functions as F

df_flat = df.select(
    F.col("repo_id"),
    F.col("uwi.uwi").alias("uwi"),
    F.col("uwi.wsn").alias("wsn"),
    # F.col("logdata.digits").alias("digits"),
    F.col("logdata.maxval").alias("max_val"),
    F.col("logdata.minval").alias("min_val"),
    F.col("logdata.mean").alias("mean"),
    F.col("logdata.stddev").alias("std_dev"),
    F.col("logdata.numpts").alias("num_points"),
    F.col("logdata.start").alias("start"),
    F.col("logdata.stop").alias("stop"),
    F.col("logdata.step").alias("step"),
    F.col("logdata.remarks").alias("remarks"),
    F.col("logdata.source").alias("source"),
    F.col("logdatax.adddate").alias("app_row_created"),
    F.col("logdatax.chgdate").alias("app_row_changed"),
    F.col("logdef.desc").alias("description"),
    F.col("logdef.logname").alias("log_name"),
    F.col("logdef.units").alias("units")
)

date_columns = [
    "app_row_created",
    "app_row_changed"
]

df_vector_log = df_flat


# enforce timestamp for dates
for col_name in date_columns:
    df_vector_log = string_to_iso_date(df_vector_log, col_name, col_name)

# add id hash
id_columns = ["repo_id", "uwi"]
df_vector_log = generate_hash(df_vector_log, "id", "vector_log", *id_columns)


display(df_vector_log)